In [2]:
# Run 'source ~/.profile'
import satpy
from satpy import Scene, find_files_and_readers
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyresample import geometry
from pyproj import CRS
from satpy.composites import GenericCompositor
from satpy.writers import to_image
import os

import datetime

sys.path.insert(0,'/home/cameron/Projects/')

files_1 = find_files_and_readers(base_dir="/home/cameron/Dokumenter/Data/combined", 
                                 reader='hypso1_l1a_nc', 
                                 start_time=datetime.datetime(2023, 3, 1, 15, 59))


files_1 = find_files_and_readers(base_dir="/home/cameron/Dokumenter/Data/combined", 
                                 reader='hypso1_l1a_nc', 
                                 start_time=datetime.datetime(2023, 3, 1, 15, 0),
                                 end_time=datetime.datetime(2023, 3, 1, 16, 0))

files_1 = find_files_and_readers(base_dir="/home/cameron/Dokumenter/Data/combined", 
                                 reader='hypso1_l1a_nc', 
                                 start_time=datetime.datetime(2023, 3, 1, 15, 0),
                                 filter_parameters={'target': 'erie'})

file_1 = ['/home/cameron/Dokumenter/Data/combined/lakeerie_2023-05-17_1553Z-l1a.nc',
          '/home/cameron/Dokumenter/Data/combined/lakeerie_2023-05-17_1553Z-bin3.points']

capture_directory = '/home/cameron/Dokumenter/Data/erie'


#from glob import glob
#scene_1 = Scene(filenames=glob('/home/cameron/Dokumenter/Data/svalbardeidembukta/svalbardeidembukta_2023-03-25_*'), reader='hypso1_l1a_nc')

In [22]:
'erie_2022-07-09_1612Z-l1a.nc'.rsplit('-', maxsplit=1)

['erie_2022-07-09_1612Z', 'l1a.nc']

In [33]:
[os.path.join(capture_directory, file) for file in os.listdir(capture_directory)]

['/home/cameron/Dokumenter/Data/erie/20220827_CaptureDL_00_erie_2022_08_27T16_05_36',
 '/home/cameron/Dokumenter/Data/erie/erie_2022-07-19_1550Z-bin3.points',
 '/home/cameron/Dokumenter/Data/erie/erie_2022-07-09_1612Z-l1a.nc',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-06-17_1541Z-l1a.nc',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-09-15_1610Z-l1a.nc',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-02-02_1552Z-l1a.nc',
 '/home/cameron/Dokumenter/Data/erie/20230605_CaptureDL_erie_2023-06-03_1612Z',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-08-20_1538Z-bin3.points',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-05-09_1622Z-l1a.nc',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-06-03_1612Z-bin3.points',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-08-24_1605Z-l1a.nc',
 '/home/cameron/Dokumenter/Data/erie/20230519_CaptureDL_erie_2023-05-17_1553Z',
 '/home/cameron/Dokumenter/Data/erie/erie_2023-04-27_1616Z-l1a.nc',
 '/home/cameron/Dokumenter/Data/erie/erie_2022-08-27_1605Z-

In [61]:
# Filter files ending in '-l1a.nc' or '-bin3.points'
#l1a_files = [file for file in os.listdir(capture_directory) if file.endswith('l1a.nc')]
#bin3_files = [file for file in os.listdir(capture_directory) if file.endswith('bin3.points')]

l1a_files = [file for file in os.listdir(capture_directory) if file.endswith('l1a.nc')]
bin3_files = [file for file in os.listdir(capture_directory) if file.endswith('bin3.points')]

l1a_files = [os.path.join(capture_directory, file) for file in os.listdir(capture_directory) if file.endswith('l1a.nc')]
bin3_files = [os.path.join(capture_directory, file) for file in os.listdir(capture_directory) if file.endswith('bin3.points')]

#file_paths = [os.path.join(capture_directory, file) for file in os.listdir(capture_directory) if os.path.isfile(os.path.join(capture_directory, file))]

# Match pairs of files based on the first part of their file names
matched_pairs = []
for l1a_file in l1a_files:
    l1a_prefix = l1a_file.rsplit('-', maxsplit=1)[0]
    for bin3_file in bin3_files:
        bin3_prefix = bin3_file.rsplit('-', maxsplit=1)[0]
        if l1a_prefix == bin3_prefix:
            l1a_file_prefix = l1a_prefix.split('/')[-1]
            matched_pairs.append((l1a_file, bin3_file, l1a_file_prefix))
            bin3_files.remove(bin3_file)  # Avoid duplicate matches

# Print the matched pairs
for pair in matched_pairs:
    print(pair)

('/home/cameron/Dokumenter/Data/erie/erie_2023-06-17_1541Z-l1a.nc', '/home/cameron/Dokumenter/Data/erie/erie_2023-06-17_1541Z-bin3.points', 'erie_2023-06-17_1541Z')
('/home/cameron/Dokumenter/Data/erie/erie_2022-08-27_1605Z-l1a.nc', '/home/cameron/Dokumenter/Data/erie/erie_2022-08-27_1605Z-bin3.points', 'erie_2022-08-27_1605Z')
('/home/cameron/Dokumenter/Data/erie/erie_2023-10-02_1559Z-l1a.nc', '/home/cameron/Dokumenter/Data/erie/erie_2023-10-02_1559Z-bin3.points', 'erie_2023-10-02_1559Z')
('/home/cameron/Dokumenter/Data/erie/erie_2023-06-04_1557Z-l1a.nc', '/home/cameron/Dokumenter/Data/erie/erie_2023-06-04_1557Z-bin3.points', 'erie_2023-06-04_1557Z')
('/home/cameron/Dokumenter/Data/erie/erie_2022-07-19_1550Z-l1a.nc', '/home/cameron/Dokumenter/Data/erie/erie_2022-07-19_1550Z-bin3.points', 'erie_2022-07-19_1550Z')
('/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-l1a.nc', '/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-bin3.points', 'erie_2023-03-01_1559Z')
('/home/ca

In [65]:
def process_capture(nc_file, points_file, name):

    files = [nc_file, points_file]

    print(files)

    scene = Scene(filenames=files, reader='hypso1_l1a_nc')
    datasets = scene.available_dataset_names()
    scene.load(datasets)
    #scene.load(['latitude', 'longitude', '80', '40', '15'])

    area_def = get_area(scene)

    resampled_scene = scene.resample(area_def, resampler='bilinear', fill_value=np.NaN)

    write_composites(scene, resampled_scene, name, gamma=2)

def get_area(scene):

    grid_lats = scene['80'].attrs['area'].lats.data
    grid_lons = scene['80'].attrs['area'].lons.data   

    # Is there a function that can do this? Possibly in pyresample.
    lon_min = grid_lons.min()
    lon_max = grid_lons.max()
    lat_min = grid_lats.min()
    lat_max = grid_lats.max()

    bbox = (lon_min,lat_min,lon_max,lat_max)
    #bbox = (-83.534546,41.356196,-82.359009,42.706660) # W. Lake Erie
    bbox = (-83.534546,41.356196,-81.359009,42.706660) # W. Lake Erie

    print(bbox)

    area_id = 'roi'
    proj_id = 'roi'
    description = 'roi'
    projection = CRS.from_epsg(4326)
    width = 750
    height = 750
    area_extent = list(bbox)

    # Define area definition
    area_def = geometry.AreaDefinition(area_id, proj_id, description, projection,  width, height, area_extent)

    return area_def

def write_composites(scene, resampled_scene, name, gamma=2):

    # Original capture composite
    s = scene
    compositor = GenericCompositor("overview")
    composite = compositor([s['80'][:,::3], s['40'][:,::3], s['15'][:,::3]]) # Red, Green, Blue
    #composite = composite[:,:,::-1] # correct for composite mirroring
    img = to_image(composite[:,:,::-1]) 
    img.invert([False, False, False])
    img.stretch("linear")
    img.gamma([gamma, gamma, gamma])
    img.save('./composites/' + name + '.png')

    # Resampled capture composites
    s = resampled_scene
    compositor = GenericCompositor("overview")
    composite = compositor([s['80'], s['40'], s['15']]) # Red, Green, Blue
    img = to_image(composite)
    img.invert([False, False, False])
    img.stretch("linear")
    img.gamma([gamma, gamma, gamma])
    img.save('./composites/resampled_' + name + '.png')


In [67]:
for pair in matched_pairs:

    nc_file = pair[0]
    points_file = pair[1]
    name = pair[2]

    process_capture(nc_file=nc_file, points_file=points_file, name=name)

['/home/cameron/Dokumenter/Data/erie/erie_2023-06-17_1541Z-l1a.nc', '/home/cameron/Dokumenter/Data/erie/erie_2023-06-17_1541Z-bin3.points']
(-83.534546, 41.356196, -81.359009, 42.70666)


/home/cameron/miniforge3/lib/python3.10/site-packages/pyproj/crs/crs.py:1286: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/cameron/miniforge3/lib/python3.10/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


In [58]:
matched_pairs[0][1]

'/home/cameron/Dokumenter/Data/erie/erie_2023-06-17_1541Z-bin3.points'

In [ ]:
list(matched_pairs[0][1], matched_pairs[0][1])